In [1]:
!pip install transformers accelerate optimum
#https://huggingface.co/docs/optimum/bettertransformer/tutorials/convert

     |████████████████████████████████| 6.3 MB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 199 kB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 227 kB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 757 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.6 MB 7.8 MB/s eta 0:00:01     |██████████████████▉             | 4.5 MB 5.3 MB/s eta 0:00:01
     |████████████████████████████████| 190 kB 6.7 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 6.5 MB/s eta 0:00:01     |██████▎                         | 1.3 MB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 2.2 MB/s eta 0:00:011
     |████████████████████████████████| 1.0 MB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 3.1 MB/s eta 0:00:011
     |████████████████████████████████| 532 kB 5.8 MB/s eta 0:00:01


In [2]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     |████████████████████████████████| 4.2 MB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 1978.0 MB 2.3 kB/s eta 0:00:012 |                                | 3.5 MB 6.1 MB/s eta 0:05:26     |                                | 5.8 MB 6.1 MB/s eta 0:05:26     |▎                               | 18.2 MB 5.8 MB/s eta 0:05:40     |▍                               | 23.0 MB 4.0 MB/s eta 0:08:08     |▍                               | 26.1 MB 4.0 MB/s eta 0:08:07     |█▍                              | 87.5 MB 6.0 MB/s eta 0:05:16     |██                              | 126.3 MB 5.8 MB/s eta 0:05:21     |██▏                             | 132.2 MB 5.8 MB/s eta 0:05:20     |██▍                             | 149.3 MB 5.4 MB/s eta 0:05:40     |██▊                             | 166.5 MB 7.2 MB/s eta 0:04:12     |██▊                             | 171.5 MB 5.4 MB/s eta 0:05:33     |███▏                    

In [10]:
# import python directory
from transformers import pipeline
from tqdm import tqdm
from glob import glob
import os
from optimum.bettertransformer import BetterTransformer
from transformers import AutoModelForSequenceClassification, AutoModel
import pandas as pd


In [11]:
#bart_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli')
#https://huggingface.co/facebook/bart-large-mnli
# set model to GPU https://huggingface.co/docs/optimum/bettertransformer/tutorials/convert
#model = bart_model.to(0) # or model.to("cuda:0")
#model = BetterTransformer.transform(bart_model)

In [4]:
# or this should work, too
classifier = pipeline("zero-shot-classification", model='facebook/bart-large-mnli', accelerator="bettertransformer", device=0)

In [5]:
input_directory, output_directory = '/workspace/dataset22/', '/workspace/dataset22/'

In [24]:
# load image text
# We use some very baseline method to get a textual representation: we just use the text of the pages that contain the image.
def load_image_text(image_id):
    ret = ''
    for txt_file in glob(input_directory +'/images/' + image_id[:3] + '/' + image_id + '/*/*/*/text.txt'):
        ret += '\n\n' + open(txt_file).read()
    return ret.strip()

def load_queries():
    file_name = input_directory + 'topics.xml'
    
    if not os.path.exists(file_name):
        raise ValueError(f'Could not find the file "{file_name}". Got: {glob(input_directory + "/*")}')
    
    topics = pd.read_xml(file_name)
    
    ret = pd.DataFrame()
    # https://github.com/terrier-org/pyterrier/issues/62\n",
    ret['query'] = topics['title'].apply(lambda i: "".join([x if x.isalnum() else " " for x in i]))
    ret['qid'] = topics['number']
    return ret


def determine_stance(text, query):
    labels = ["contra "+ query, "pro "+ query, "neutral "+ query]
    result = classifier(text, labels)
    # sorts the labels starting with the most likely
    if result["labels"][0].startswith("contra"):
        return "contra", result["scores"][0]
    elif result["labels"][0].startswith("pro"):
        return "pro", result["scores"][0]
    else: 
        return "neutral", result["scores"][0]
    

def detect_stance_for_image_ids_and_return_best_x_results_for_each_side(image_ids, query, x):
    # get image ids from preranking
    # get image texts based on preranking 
    pro_imgs = {}
    contra_imgs = {}
    pro_ids = []
    contra_ids = []
    
    for image_id in image_ids:
        text = load_image_text(image_id)
        # deploy model
        stance, score = determine_stance(text, query)
        print(image_id, stance, score)
        # separate pro and con texts; sort out neutral texts
        if stance == "pro":
            pro_imgs[image_id] = score
        elif stance == "contra": 
            contra_imgs[image_id] = score
            
    # sort according to scores
    sorted_pro = dict(sorted(pro_imgs.items(), key=lambda x:x[1], reverse=True))
    sorted_contra = dict(sorted(contra_imgs.items(), key=lambda x:x[1], reverse=True))
    # return best x results for each side
    i = 0
    for key in sorted_pro:
        if i < len(sorted_pro.keys()) and i < x:
            pro_ids.append(key)
        else: 
            break
        i += 1
    i = 0
    for key in sorted_contra:
        if i < len(sorted_contra.keys()) and i < x:
            contra_ids.append(key)
        else: 
            break
        i += 1
    return pro_ids, contra_ids, sorted_pro, sorted_contra
# not sorted right


In [14]:
queries = load_queries()  
queries

,query,qid
0,Should teachers get tenure,1
1,Is vaping with e cigarettes safe,2
2,Should insider trading be allowed,3
3,Should corporal punishment be used in schools,4
4,Should social security be privatized,5
5,Is a college education worth it,6
6,Should felons who have completed their sentenc...,7
7,Should abortion be legal,8
8,Should students have to wear school uniforms,9
9,Should any vaccines be required for children,10


In [18]:
test_ids = os.listdir(input_directory +'/images/' + "I00")[0:50] #["I00", "I01","I02", "I03", "I04", "I05", "I06"]
query = ""

In [20]:
%%time

detect_stance_for_image_ids_and_return_best_x_results_for_each_side(test_ids, query, 10)

I000330ba4ea0ad13 contra 0.467972993850708
I000d8de9c4746ee9 pro 0.3815426826477051
I0010d5b5473065fa pro 0.604080080986023
I001652ec040f07c4 pro 0.43602487444877625
I001a318b3176d501 pro 0.37916141748428345
I001b3068979b8629 pro 0.4018155038356781
I001c2de5ef28f77f pro 0.47396448254585266
I001d567f0fb443d7 neutral 0.3531552255153656
I00246a6916fec768 pro 0.3388686180114746
I0024c0097c425855 pro 0.3830471634864807
I002747d3611b43da pro 0.4405139088630676
I0027bcd7cd6938d9 pro 0.4139585494995117
I00290a8d0fe34013 pro 0.48053017258644104
I002959a3418da38a pro 0.41091617941856384
I002d16483c9a4e26 contra 0.44968870282173157
I002dd0599859cb69 contra 0.39932355284690857
I002e942703b3e786 pro 0.4002256989479065
I003811155228eb5d pro 0.4027808904647827
I003a0ad4cc3bae04 pro 0.4484193027019501
I003afc8037f37ff2 pro 0.4007573425769806
I003cedcf82e2f803 pro 0.4110259413719177
I00461cb2a8d80b57 pro 0.38769176602363586
I004cdffb8811bc94 pro 0.42624548077583313
I004d01dd39c8e72c pro 0.3489230871200

(['I0078f898366817c3',
  'I006ed234fb94b887',
  'I0010d5b5473065fa',
  'I007def293fc43028',
  'I005f18e15dbe8263',
  'I00a2593a7644b4d7',
  'I0083235a39d0f3da',
  'I007a0f94f819667a',
  'I00290a8d0fe34013',
  'I001c2de5ef28f77f'],
 ['I0099831c83bd7324',
  'I000330ba4ea0ad13',
  'I002d16483c9a4e26',
  'I005fc4390af43423',
  'I002dd0599859cb69',
  'I00a534edc86bf5cd'])

In [13]:
sequence_to_classify = "Abortion is a universal right."
candidate_labels = ['pro abortion', 'contra abortion', 'neutral abortion']
classifier(sequence_to_classify, candidate_labels)


{'sequence': 'Abortion is a universal right.',
 'labels': ['pro abortion', 'neutral abortion', 'contra abortion'],
 'scores': [0.8498501777648926, 0.1026904508471489, 0.04745934531092644]}